In [7]:
import re
def chunk_sentences_with_length_limit(text, max_length):
    sentences = re.split(r"(?<=[.!?])\s+", text)
    chunks, current_chunk = [], ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_length:
            current_chunk += " " + sentence if current_chunk else sentence
        else:
            chunks.append(current_chunk)
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

In [8]:
def preprocess_text_for_llm(input_text):
    """
    Preprocesses text data for LLM by simplifying and structuring it.

    Args:
        input_text (str): Raw input text.

    Returns:
        str: Preprocessed text suitable for LLM input.
    """
    # Step 1: Remove unnecessary line breaks
    text = re.sub(r"\n+", " ", input_text)

    # Step 2: Simplify formatting (remove redundant symbols and special cases)
    text = re.sub(r"\*\*|`|\"|\(.*?\)|—>", "", text)

    # Step 3: Structure the text
    sections = []
    current_section = ""

    for line in text.split(". "):
        line = line.strip()
        if line.startswith("###"):
            # Append previous section if exists
            if current_section:
                sections.append(current_section.strip())
            # Start a new section
            current_section = f"\n{line.strip('#').strip()}\n"
        else:
            current_section += f"{line.strip()} "

    # Append the last section
    if current_section:
        sections.append(current_section.strip())

    # Step 4: Final formatting of structured sections
    processed_text = "\n".join(sections).strip()

    # Step 5: Add bullet points and reformat lists for LLM readability
    processed_text = re.sub(r"- ", "\u2022 ", processed_text)

    return processed_text

In [9]:
def load_data(file_path: str):
    with open(file_path, "r", encoding="utf-8") as file:
        data = file.read()
    return data

In [13]:
import re

text = load_data("raw_data.txt")
#text = preprocess_text_for_llm(test_data)
#print(text)
max_length=50
sentences = re.split(r"(?<=[.!?])\s+", text)
chunks, current_chunk = [], ""
for sentence in sentences:
    if len(current_chunk) + len(sentence) + 1 <= max_length:
        current_chunk += " " + sentence if current_chunk else sentence
    else:
        chunks.append(current_chunk)
        current_chunk = sentence
if current_chunk:
    chunks.append(current_chunk)


**인터넷**은 end system에서 수행되는 분산 애플리케이션에게 서비스를 제공해주는 **인프라 구조**이다.

이상적으로는 인터넷 서비스가 데이터 손실 없이 즉시 두 종단 시스템 간에 원하는 만큼의 데이터를 이동시키기를 원한다.

하지만 현실에서는 어려우며 컴퓨터 네트워크는 두 종단 시스템 간에 전달 될 수 있는 초당 데이터의양 —> 처리율을 제한한다.

따라서 이로이해 end system 사이에는 지연이 발생하며 패킷을 잃어버리기도 한다.

### 1. 패킷 교환 네트워크에서의 지연

패킷은 출발지 → 라우터들 → 도착지 과정을 거친다.

패킷이 이런 경로를 따라 한 노드에서 다음 노드로 전달될 때 패킷은 경로상의 **각 노드에서 다양한 지연**을 겪게 되며 이들은 쌓여서 **전체 노드 지연**을 일으킨다.

- 노드 처리 지연 (nodal processing delay)
- 큐잉 지연 (queueing delay)
- 전송 지연 ( transmission delay )
- 전파 지연 ( propagation delay )


이 그림은 라우터 A에서 노드 지연을 나타낸 것이다.

출발지와 목적지 사이 종단 간 경로의 일부로서, 한 패킷이 업스트림 노드로부터 라우터 A를 통해 라우터 B로 보내진다.

- 업스트림 : 클라이언트에서 서버로 전송되는 데이터의 흐름
- 다운 스트림 : 서버에서 클라이언트로 전송되는 데이터의 흐름, 일반적으로 다운스트림  트래픽은 업스트림 트래픽보다 더 많은 볼륨이 있다.

라우터 A는 라우터 B에 이르는 `하나의 출력(outgoing) 링크`를 가지며, 이 링크 앞에 `큐(queue, 버퍼(buffer))`가 존재한다.

1. 패킷이 업스트림 노드로부터 라우터 A에 도착한다.
2. 라우터 A는 그 패킷에 대한 적당한 출력 링크를 결정하기 위해 패킷 헤더를 조사한다.
3. 라우터 A는 선택된 링크로 그 패킷을 보낸다. (그림에서 선택된 링크 = 라우터 B에 이르는 링크)

**Processing Delay ( 처리 지연 )**